In [ ]:
print('Setup complete.')

# Human-in-the-Loop & AI Decision Making Demo with AskSage

## Focus: AI-Powered Approval Gates with GPT-5-Mini

This notebook demonstrates real AI integration for human oversight in automated systems using AskSage with GPT-5-Mini:
- **AI-Powered Risk Assessment**: Using GPT-5-Mini to evaluate action risks and confidence levels
- **Intelligent Approval Gates**: AI-driven decision making with human oversight
- **Smart Stop/Ask Thresholds**: Dynamic confidence evaluation using large language models
- **AI-Enhanced Acceptance Criteria**: Automated business rule validation with natural language understanding

---

In [ ]:
# Install required packages
!pip install asksageclient pandas matplotlib seaborn plotly ipywidgets

# All required imports for this notebook
import json
import time
import os
import sys
from typing import Dict, List, Any, Optional
from dataclasses import dataclass, asdict
from enum import Enum
from datetime import datetime
from abc import ABC, abstractmethod

# AskSage client for AI integration
from asksageclient import AskSageClient

print("🤖 AI-Powered Human-in-the-Loop Demo Environment Setup Complete")
print("🔧 Ready to demonstrate AI-enhanced approval gates with GPT-5-Mini")

## 1. AskSage Client Setup for GPT-5-Mini

Configure the AskSage client to use GPT-5-Mini for AI-powered decision making.

In [ ]:
# AskSage Client Configuration
# NOTE: You need to set up your AskSage credentials
# Create a credentials.json file or set environment variables

# Option 1: Using environment variables (recommended)
ASKSAGE_API_KEY = os.getenv('ASKSAGE_API_KEY')
ASKSAGE_BASE_URL = os.getenv('ASKSAGE_BASE_URL', 'https://api.asksage.ai')

# Option 2: Direct configuration (for demo purposes)
# ASKSAGE_API_KEY = "your_api_key_here"
# ASKSAGE_BASE_URL = "https://api.asksage.ai"

if not ASKSAGE_API_KEY:
    print("⚠️ WARNING: ASKSAGE_API_KEY not found in environment variables")
    print("Please set your AskSage API key to use GPT-5-Mini integration")
    ASKSAGE_API_KEY = "demo_key_placeholder"  # For demo structure

# Initialize AskSage client
try:
    ask_sage_client = AskSageClient(
        api_key=ASKSAGE_API_KEY,
        base_url=ASKSAGE_BASE_URL
    )
    print("✅ AskSage client initialized successfully")
    print("🎯 Target Model: GPT-5-Mini for AI decision making")
except Exception as e:
    print(f"❌ Error initializing AskSage client: {e}")
    print("📝 Continuing with demo structure (AI features will be simulated)")

## 2. Enhanced Risk Assessment Framework with AI

Build a framework that uses GPT-5-Mini to intelligently assess action risks and confidence levels.

In [ ]:
class RiskLevel(Enum):
    LOW = "low"
    MEDIUM = "medium"
    HIGH = "high"
    CRITICAL = "critical"

class ActionStatus(Enum):
    PENDING = "pending"
    APPROVED = "approved"
    DENIED = "denied"
    EXPIRED = "expired"

@dataclass
class ActionRequest:
    id: str
    description: str
    proposed_action: Dict[str, Any]
    timestamp: str
    context: str = ""
    ai_risk_assessment: Optional[Dict[str, Any]] = None
    ai_confidence: Optional[float] = None
    risk_level: Optional[RiskLevel] = None
    status: ActionStatus = ActionStatus.PENDING
    approval_reason: Optional[str] = None
    approver: Optional[str] = None

def query_gpt5_mini(prompt: str, system_prompt: str = None) -> str:
    """
    Query GPT-5-Mini through AskSage for AI decision making.
    """
    try:
        response = ask_sage_client.query(
            message=prompt,
            model="gpt-5-mini",  # Specifically using GPT-5-Mini as requested
            system_prompt=system_prompt,
            temperature=0.1  # Low temperature for consistent decision making
        )
        
        if isinstance(response, dict) and 'response' in response:
            return response['response']
        return str(response)
    except Exception as e:
        print(f"⚠️ GPT-5-Mini query failed: {e}")
        return f"AI_ERROR: {str(e)}"

def ai_assess_action_risk(request: ActionRequest) -> ActionRequest:
    """
    Use GPT-5-Mini to assess the risk level and confidence of an action request.
    """
    system_prompt = """
You are an expert AI risk assessment system. Analyze the provided action request and return a JSON response with:
1. risk_level: one of ["low", "medium", "high", "critical"]
2. confidence: a float between 0.0 and 1.0
3. reasoning: a brief explanation of your assessment
4. potential_impacts: list of potential positive and negative impacts
5. mitigation_suggestions: list of risk mitigation recommendations

Consider factors like:
- Data sensitivity and scope of impact
- Reversibility of the action
- System criticality
- Security implications
- Business continuity impact

Respond only with valid JSON.
"""
    
    prompt = f"""
Assess this action request:
ID: {request.id}
Description: {request.description}
Proposed Action: {json.dumps(request.proposed_action, indent=2)}
Context: {request.context}
Timestamp: {request.timestamp}

Provide risk assessment as JSON:
"""
    
    ai_response = query_gpt5_mini(prompt, system_prompt)
    
    try:
        # Parse AI response as JSON
        ai_assessment = json.loads(ai_response)
        
        # Update request with AI assessment
        request.ai_risk_assessment = ai_assessment
        request.ai_confidence = ai_assessment.get('confidence', 0.5)
        request.risk_level = RiskLevel(ai_assessment.get('risk_level', 'medium'))
        
    except (json.JSONDecodeError, ValueError) as e:
        print(f"⚠️ AI response parsing failed: {e}")
        # Fallback to simulated assessment
        request.ai_risk_assessment = {
            "risk_level": "medium",
            "confidence": 0.7,
            "reasoning": "Fallback assessment due to AI parsing error",
            "error": str(e)
        }
        request.ai_confidence = 0.7
        request.risk_level = RiskLevel.MEDIUM
    
    return request

# Example action requests for testing
sample_requests = [
    ActionRequest(
        id="req_001",
        description="Delete temporary log files older than 7 days",
        proposed_action={"command": "rm", "path": "/var/log/temp/*", "age": "7d", "pattern": "*.log"},
        context="Routine maintenance to free up disk space on development server",
        timestamp=datetime.now().isoformat()
    ),
    ActionRequest(
        id="req_002",
        description="Update production database schema to add new user preferences table",
        proposed_action={
            "operation": "CREATE TABLE", 
            "table": "user_preferences", 
            "database": "production_users",
            "backup_required": True
        },
        context="Adding new feature for user customization settings",
        timestamp=datetime.now().isoformat()
    ),
    ActionRequest(
        id="req_003",
        description="Deploy hotfix to resolve critical security vulnerability",
        proposed_action={
            "operation": "deploy",
            "target": "production",
            "version": "v2.1.3-hotfix",
            "rollback_plan": "automated"
        },
        context="Emergency deployment to patch CVE-2024-12345 discovered in authentication module",
        timestamp=datetime.now().isoformat()
    )
]

print("🤖 AI Risk Assessment Framework Initialized")
print("📝 Sample requests created for GPT-5-Mini evaluation")
for req in sample_requests:
    print(f"  🎯 {req.id}: {req.description}")

## 3. AI-Powered Risk Assessment in Action

Let GPT-5-Mini evaluate each action request and provide intelligent risk assessments.

In [ ]:
print("🔍 Running AI Risk Assessment with GPT-5-Mini...\n")

assessed_requests = []
for request in sample_requests:
    print(f"🤖 Analyzing {request.id}: {request.description}")
    
    # Get AI risk assessment from GPT-5-Mini
    assessed_request = ai_assess_action_risk(request)
    assessed_requests.append(assessed_request)
    
    # Display results
    print(f"   🎯 AI Risk Level: {assessed_request.risk_level.value.upper()}")
    print(f"   📊 AI Confidence: {assessed_request.ai_confidence:.1%}")
    
    if assessed_request.ai_risk_assessment:
        reasoning = assessed_request.ai_risk_assessment.get('reasoning', 'N/A')
        print(f"   💭 AI Reasoning: {reasoning}")
        
        impacts = assessed_request.ai_risk_assessment.get('potential_impacts', [])
        if impacts:
            print(f"   ⚡ Potential Impacts: {', '.join(impacts[:2])}...")
    
    print("   " + "="*50)
    print()

print("✅ AI Risk Assessment Complete")
print(f"📈 Processed {len(assessed_requests)} requests using GPT-5-Mini")

## 4. Dynamic Threshold Configuration with AI Insights

Configure intelligent thresholds that adapt based on AI confidence and risk assessment patterns.

In [ ]:
@dataclass
class AIThresholdConfig:
    risk_level: RiskLevel
    auto_approve_threshold: float  # AI confidence level for automatic approval
    ask_threshold: float          # AI confidence level that requires human input
    deny_threshold: float         # Below this AI confidence, automatically deny
    ai_reasoning_required: bool   # Whether AI reasoning is required for decisions

# AI-enhanced threshold configurations
AI_THRESHOLD_CONFIGS = {
    RiskLevel.LOW: AIThresholdConfig(
        risk_level=RiskLevel.LOW,
        auto_approve_threshold=0.85,  # Lower threshold due to AI analysis
        ask_threshold=0.65,
        deny_threshold=0.40,
        ai_reasoning_required=False
    ),
    RiskLevel.MEDIUM: AIThresholdConfig(
        risk_level=RiskLevel.MEDIUM,
        auto_approve_threshold=0.90,
        ask_threshold=0.75,
        deny_threshold=0.50,
        ai_reasoning_required=True
    ),
    RiskLevel.HIGH: AIThresholdConfig(
        risk_level=RiskLevel.HIGH,
        auto_approve_threshold=0.95,
        ask_threshold=0.80,
        deny_threshold=0.60,
        ai_reasoning_required=True
    ),
    RiskLevel.CRITICAL: AIThresholdConfig(
        risk_level=RiskLevel.CRITICAL,
        auto_approve_threshold=0.99,  # Nearly impossible to auto-approve
        ask_threshold=0.85,
        deny_threshold=0.70,
        ai_reasoning_required=True
    )
}

def ai_evaluate_action_requirement(request: ActionRequest) -> str:
    """
    Use AI assessment to determine if an action requires approval.
    """
    if not request.risk_level or request.ai_confidence is None:
        return "AI_ASSESSMENT_REQUIRED"
    
    config = AI_THRESHOLD_CONFIGS[request.risk_level]
    
    # Check if AI reasoning is available when required
    if config.ai_reasoning_required:
        if not request.ai_risk_assessment or not request.ai_risk_assessment.get('reasoning'):
            return "AI_REASONING_REQUIRED"
    
    # Evaluate based on AI confidence
    if request.ai_confidence >= config.auto_approve_threshold:
        return "AUTO_APPROVE"
    elif request.ai_confidence >= config.ask_threshold:
        return "ASK_HUMAN"
    elif request.ai_confidence >= config.deny_threshold:
        return "ASK_HUMAN"  # Low confidence but not below deny threshold
    else:
        return "AUTO_DENY"

def generate_ai_approval_prompt(request: ActionRequest) -> str:
    """
    Generate an intelligent approval prompt using AI insights.
    """
    system_prompt = """
You are an AI assistant helping with human approval decisions. 
Generate a clear, concise approval prompt that highlights key decision factors.
Include risk summary, confidence level, and specific recommendations.
"""
    
    prompt = f"""
Generate an approval prompt for this action:
ID: {request.id}
Description: {request.description}
AI Risk Level: {request.risk_level.value if request.risk_level else 'Unknown'}
AI Confidence: {request.ai_confidence:.1%} if request.ai_confidence else 'Unknown'
AI Assessment: {json.dumps(request.ai_risk_assessment, indent=2) if request.ai_risk_assessment else 'Not available'}

Create a concise human-readable approval prompt.
"""
    
    ai_prompt = query_gpt5_mini(prompt, system_prompt)
    return ai_prompt

print("⚙️ AI-Enhanced Threshold Configuration:")
for risk_level, config in AI_THRESHOLD_CONFIGS.items():
    reasoning_req = "✓" if config.ai_reasoning_required else "✗"
    print(f"  {risk_level.value.upper()}: Auto≥{config.auto_approve_threshold:.0%}, Ask≥{config.ask_threshold:.0%}, Deny<{config.deny_threshold:.0%}, AI Reasoning: {reasoning_req}")

print("\n🔍 Evaluating Sample Requests with AI Decision Logic:")
for req in assessed_requests:
    decision = ai_evaluate_action_requirement(req)
    confidence_str = f"{req.ai_confidence:.0%}" if req.ai_confidence else "N/A"
    risk_str = req.risk_level.value.upper() if req.risk_level else "N/A"
    print(f"  {req.id}: {decision} (AI Risk: {risk_str}, AI Confidence: {confidence_str})")

## 5. AI-Enhanced Acceptance Criteria

Implement acceptance tests that leverage GPT-5-Mini's natural language understanding for intelligent business rule validation.

In [ ]:
class AIAcceptanceCriterion(ABC):
    """Base class for AI-enhanced acceptance criteria tests."""
    
    def __init__(self, name: str, description: str, use_ai: bool = True):
        self.name = name
        self.description = description
        self.use_ai = use_ai
    
    @abstractmethod
    def evaluate(self, request: ActionRequest) -> bool:
        """Return True if the criterion is met."""
        pass
    
    def ai_evaluate(self, request: ActionRequest, rule_description: str) -> Dict[str, Any]:
        """Use GPT-5-Mini to evaluate complex business rules."""
        system_prompt = f"""
You are an AI system evaluating business rules for action approval.
Rule: {rule_description}

Analyze the action request and return JSON with:
- passes: boolean (true if rule is satisfied)
- confidence: float 0.0-1.0 (how confident you are)
- reasoning: string explaining your evaluation
- concerns: list of any concerns or edge cases identified

Respond only with valid JSON.
"""
        
        prompt = f"""
Evaluate this action against the business rule:

Action: {request.description}
Details: {json.dumps(request.proposed_action, indent=2)}
Context: {request.context}
Risk Level: {request.risk_level.value if request.risk_level else 'Unknown'}

Does this action satisfy the business rule?
"""
        
        ai_response = query_gpt5_mini(prompt, system_prompt)
        
        try:
            return json.loads(ai_response)
        except json.JSONDecodeError:
            return {
                "passes": False,
                "confidence": 0.0,
                "reasoning": "AI evaluation failed - JSON parse error",
                "concerns": ["Could not parse AI response"]
            }

class AIProductionDataProtection(AIAcceptanceCriterion):
    def __init__(self):
        super().__init__(
            "ai-prod-data-protection",
            "AI-enhanced protection against production data operations",
            use_ai=True
        )
    
    def evaluate(self, request: ActionRequest) -> bool:
        if not self.use_ai:
            # Fallback to simple pattern matching
            action = request.proposed_action
            dangerous_ops = ["rm", "DELETE", "DROP", "TRUNCATE"]
            return not any(op in str(action) for op in dangerous_ops if "prod" in str(action).lower())
        
        rule = "Prevent any destructive operations (delete, drop, truncate) on production data without explicit safeguards and approval workflows."
        ai_result = self.ai_evaluate(request, rule)
        return ai_result.get('passes', False)

class AIBusinessHoursCompliance(AIAcceptanceCriterion):
    def __init__(self):
        super().__init__(
            "ai-business-hours",
            "AI-powered business hours and maintenance window compliance",
            use_ai=True
        )
    
    def evaluate(self, request: ActionRequest) -> bool:
        if not self.use_ai:
            # Simple time-based check
            current_hour = datetime.now().hour
            if request.risk_level in [RiskLevel.HIGH, RiskLevel.CRITICAL]:
                return 9 <= current_hour <= 17
            return True
        
        current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S %A")
        rule = f"High-risk operations should only be performed during business hours (9 AM - 5 PM, Monday-Friday) unless it's an emergency. Current time: {current_time}"
        ai_result = self.ai_evaluate(request, rule)
        return ai_result.get('passes', True)

class AISecurityComplianceCheck(AIAcceptanceCriterion):
    def __init__(self):
        super().__init__(
            "ai-security-compliance",
            "AI-powered security and compliance validation",
            use_ai=True
        )
    
    def evaluate(self, request: ActionRequest) -> bool:
        rule = """
Ensure the action complies with security best practices:
1. No hardcoded credentials or secrets
2. Appropriate access controls and permissions
3. Audit trail and logging requirements
4. Data privacy and protection compliance
5. Change management and approval workflows
"""
        ai_result = self.ai_evaluate(request, rule)
        return ai_result.get('passes', False)

# Define AI-enhanced acceptance test suite
AI_ACCEPTANCE_CRITERIA = [
    AIProductionDataProtection(),
    AIBusinessHoursCompliance(),
    AISecurityComplianceCheck()
]

def run_ai_acceptance_tests(request: ActionRequest) -> Dict[str, Any]:
    """
    Run all AI-enhanced acceptance criteria tests on a request.
    """
    results = {}
    all_passed = True
    ai_insights = []
    
    print(f"🧪 Running AI acceptance tests for {request.id}...")
    
    for criterion in AI_ACCEPTANCE_CRITERIA:
        try:
            passed = criterion.evaluate(request)
            
            # Get AI insights if available
            if criterion.use_ai and hasattr(criterion, '_last_ai_result'):
                ai_insight = getattr(criterion, '_last_ai_result', {})
                ai_insights.append({
                    "criterion": criterion.name,
                    "reasoning": ai_insight.get('reasoning', ''),
                    "concerns": ai_insight.get('concerns', [])
                })
            
            results[criterion.name] = {
                "passed": passed,
                "description": criterion.description,
                "ai_enhanced": criterion.use_ai
            }
            
            if not passed:
                all_passed = False
                
        except Exception as e:
            results[criterion.name] = {
                "passed": False,
                "description": criterion.description,
                "error": str(e),
                "ai_enhanced": criterion.use_ai
            }
            all_passed = False
    
    return {
        "all_passed": all_passed, 
        "criteria_results": results,
        "ai_insights": ai_insights
    }

print("🧪 AI-Enhanced Acceptance Criteria Test Suite:")
for criterion in AI_ACCEPTANCE_CRITERIA:
    ai_badge = "🤖" if criterion.use_ai else "🔧"
    print(f"  {ai_badge} {criterion.name}: {criterion.description}")

print("\n📋 Testing Sample Requests with AI Acceptance Criteria:")
for req in assessed_requests:
    test_results = run_ai_acceptance_tests(req)
    status = "✅ PASS" if test_results["all_passed"] else "❌ FAIL"
    print(f"\n  {req.id}: {status}")
    
    for name, result in test_results["criteria_results"].items():
        icon = "✓" if result["passed"] else "✗"
        ai_badge = "🤖" if result.get("ai_enhanced") else "🔧"
        print(f"    {icon} {ai_badge} {name}")
        
        if "error" in result:
            print(f"      ⚠️ Error: {result['error']}")

## 6. Complete AI-Powered Approval Workflow

Integrate all components into a comprehensive workflow that leverages GPT-5-Mini for intelligent decision making.

In [ ]:
class AIApprovalWorkflow:
    def __init__(self):
        self.pending_requests = {}
        self.completed_requests = {}
        self.ai_insights_log = []
    
    def process_request(self, request: ActionRequest) -> Dict[str, Any]:
        """
        Process an action request through the complete AI-enhanced workflow.
        """
        workflow_log = []
        
        # Step 1: AI Risk Assessment
        workflow_log.append("🤖 Running AI risk assessment with GPT-5-Mini...")
        if not request.ai_risk_assessment:
            request = ai_assess_action_risk(request)
        
        risk_level = request.risk_level.value if request.risk_level else "unknown"
        confidence = request.ai_confidence or 0.0
        workflow_log.append(f"🎯 AI Assessment: {risk_level.upper()} risk, {confidence:.0%} confidence")
        
        # Step 2: AI-Enhanced Acceptance Tests
        workflow_log.append("🧪 Running AI-enhanced acceptance criteria tests...")
        acceptance_results = run_ai_acceptance_tests(request)
        
        if not acceptance_results["all_passed"]:
            request.status = ActionStatus.DENIED
            request.approval_reason = "Failed AI-enhanced acceptance criteria"
            workflow_log.append("❌ Request denied - failed AI acceptance tests")
            
            self.completed_requests[request.id] = request
            return {
                "status": "denied",
                "reason": "ai_acceptance_criteria_failed",
                "workflow_log": workflow_log,
                "acceptance_results": acceptance_results
            }
        
        workflow_log.append("✅ AI acceptance tests passed")
        
        # Step 3: AI-Powered Threshold Evaluation
        workflow_log.append("⚖️ Evaluating with AI-enhanced thresholds...")
        decision = ai_evaluate_action_requirement(request)
        
        if decision == "AUTO_APPROVE":
            request.status = ActionStatus.APPROVED
            request.approval_reason = f"Auto-approved by AI system (confidence: {confidence:.0%})"
            request.approver = "gpt-5-mini"
            workflow_log.append("✅ Auto-approved by AI system")
            
            self.completed_requests[request.id] = request
            return {
                "status": "approved",
                "reason": "ai_auto_approved",
                "workflow_log": workflow_log,
                "ai_assessment": request.ai_risk_assessment
            }
        
        elif decision == "AUTO_DENY":
            request.status = ActionStatus.DENIED
            request.approval_reason = f"Auto-denied by AI system (low confidence: {confidence:.0%})"
            workflow_log.append("❌ Auto-denied by AI system")
            
            self.completed_requests[request.id] = request
            return {
                "status": "denied",
                "reason": "ai_auto_denied",
                "workflow_log": workflow_log,
                "ai_assessment": request.ai_risk_assessment
            }
        
        # Step 4: Generate AI-Powered Human Approval Prompt
        workflow_log.append("👤 Human approval required - generating AI-powered prompt")
        ai_approval_prompt = generate_ai_approval_prompt(request)
        
        self.pending_requests[request.id] = request
        
        return {
            "status": "pending_approval",
            "reason": "human_approval_required",
            "workflow_log": workflow_log,
            "ai_approval_prompt": ai_approval_prompt,
            "ai_assessment": request.ai_risk_assessment,
            "acceptance_results": acceptance_results
        }
    
    def get_ai_insights_summary(self) -> str:
        """
        Generate a summary of AI insights from processed requests.
        """
        system_prompt = """
You are an AI system analyst. Summarize the key insights and patterns 
from the approval workflow data. Focus on:
1. Common risk patterns
2. Decision trends
3. Areas for improvement
4. Recommendations for threshold tuning

Keep the summary concise and actionable.
"""
        
        all_requests = list(self.completed_requests.values()) + list(self.pending_requests.values())
        insights_data = {
            "total_requests": len(all_requests),
            "risk_levels": [r.risk_level.value for r in all_requests if r.risk_level],
            "confidence_levels": [r.ai_confidence for r in all_requests if r.ai_confidence],
            "statuses": [r.status.value for r in all_requests]
        }
        
        prompt = f"""
Analyze this approval workflow data:
{json.dumps(insights_data, indent=2)}

Provide insights and recommendations:
"""
        
        return query_gpt5_mini(prompt, system_prompt)

# Demo the complete AI-powered workflow
ai_workflow = AIApprovalWorkflow()

print("🔄 Processing Sample Requests Through AI-Powered Workflow:\n")
for req in sample_requests:
    print(f"📝 Processing {req.id}: {req.description}")
    print(f"   Context: {req.context}")
    
    result = ai_workflow.process_request(req)
    
    print(f"   🎯 Result: {result['status'].upper()}")
    for log_entry in result['workflow_log']:
        print(f"     {log_entry}")
    
    if result['status'] == 'pending_approval':
        print("\n" + "="*70)
        print("🤖 AI-GENERATED APPROVAL PROMPT:")
        print("="*70)
        print(result['ai_approval_prompt'])
        print("="*70 + "\n")
    
    elif 'ai_assessment' in result and result['ai_assessment']:
        assessment = result['ai_assessment']
        print(f"     🧠 AI Reasoning: {assessment.get('reasoning', 'N/A')}")
        if 'mitigation_suggestions' in assessment:
            print(f"     🛡️ AI Suggestions: {', '.join(assessment['mitigation_suggestions'][:2])}")
    
    print("\n")

## 7. AI Insights and Workflow Analytics

Generate intelligent insights about the approval workflow using GPT-5-Mini's analytical capabilities.

In [ ]:
print("🧠 Generating AI Insights Summary with GPT-5-Mini...\n")

insights_summary = ai_workflow.get_ai_insights_summary()
print("📊 AI WORKFLOW ANALYTICS & INSIGHTS:")
print("="*60)
print(insights_summary)
print("="*60)

# Additional workflow statistics
all_requests = list(ai_workflow.completed_requests.values()) + list(ai_workflow.pending_requests.values())
print("\n📈 WORKFLOW STATISTICS:")
print(f"Total Requests Processed: {len(all_requests)}")
print(f"Completed: {len(ai_workflow.completed_requests)}")
print(f"Pending Human Approval: {len(ai_workflow.pending_requests)}")

if all_requests:
    risk_distribution = {}
    for req in all_requests:
        if req.risk_level:
            risk_distribution[req.risk_level.value] = risk_distribution.get(req.risk_level.value, 0) + 1
    
    print("\nRisk Level Distribution:")
    for risk, count in risk_distribution.items():
        print(f"  {risk.upper()}: {count}")
    
    avg_confidence = sum(r.ai_confidence for r in all_requests if r.ai_confidence) / len([r for r in all_requests if r.ai_confidence])
    print(f"\nAverage AI Confidence: {avg_confidence:.1%}")

print("\n✨ Demo completed successfully with GPT-5-Mini integration!")

## 8. Demo Summary & Key Takeaways

### What We've Demonstrated with GPT-5-Mini:

1. **AI-Powered Risk Assessment**: GPT-5-Mini intelligently evaluates action risks and provides confidence scores
2. **Intelligent Approval Gates**: Dynamic decision making based on AI analysis rather than static rules
3. **Smart Acceptance Criteria**: Natural language understanding for complex business rule validation
4. **AI-Enhanced Human Prompts**: Context-aware approval requests generated by AI
5. **Workflow Analytics**: AI-driven insights and recommendations for process improvement

### Key Benefits of AI Integration:

- **Adaptive Intelligence**: System learns and adapts to new patterns and contexts
- **Natural Language Processing**: Understands complex business rules in plain language
- **Context Awareness**: Considers situational factors beyond simple pattern matching
- **Explainable Decisions**: Provides reasoning and justification for each decision
- **Continuous Learning**: Can improve over time with feedback and new data

### Implementation Considerations:

- **Model Selection**: GPT-5-Mini provides excellent balance of capability and efficiency
- **Confidence Calibration**: Regular validation of AI confidence scores against actual outcomes
- **Fallback Mechanisms**: Robust error handling when AI services are unavailable
- **Human Oversight**: Maintaining appropriate human involvement in critical decisions
- **Audit Trail**: Complete logging of AI reasoning for compliance and debugging

### Next Steps:

- **Model Fine-tuning**: Train models on your specific domain and decision patterns
- **Integration Testing**: Validate AI decisions against historical data
- **Performance Monitoring**: Track AI accuracy and adjust thresholds accordingly
- **User Feedback Loop**: Incorporate human feedback to improve AI performance

---

**🚀 Ready to implement AI-powered approval workflows in your organization!**